# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [131]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt


In [132]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [133]:
spaceship.shape

(8693, 14)

**Check for data types**

In [134]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

**Check for missing values**

In [135]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [136]:
spaceship = spaceship.dropna().reset_index()

In [137]:
spaceship.isnull().sum()

index           0
PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [138]:
spaceship['Cabin']

0          B/0/P
1          F/0/S
2          A/0/S
3          A/0/S
4          F/1/S
          ...   
6601      A/98/P
6602    G/1499/S
6603    G/1500/S
6604     E/608/S
6605     E/608/S
Name: Cabin, Length: 6606, dtype: object

In [139]:
for i in range(0, len(spaceship)):
    spaceship.iloc[i, 4] = spaceship['Cabin'][i][0]

- Drop PassengerId and Name

In [140]:
columns_to_drop = ['PassengerId', 'Name']
spaceship.drop(columns=columns_to_drop, inplace=True)

- For non-numerical columns, do dummies.

In [141]:
spaceship_dummies = pd.get_dummies(spaceship, columns = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Transported'])

In [142]:
spaceship_dummies

,index,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Transported_False,Transported_True
0,0,39.0,0.0,0.0,0.0,0.0,0.0,False,True,False,...,False,False,False,False,False,True,True,False,True,False
1,1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,...,True,False,False,False,False,True,True,False,False,True
2,2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,...,False,False,False,False,False,True,False,True,True,False
3,3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,...,False,False,False,False,False,True,True,False,True,False
4,4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,...,True,False,False,False,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6601,8688,41.0,0.0,6819.0,0.0,1643.0,74.0,False,True,False,...,False,False,False,True,False,False,False,True,True,False
6602,8689,18.0,0.0,0.0,0.0,0.0,0.0,True,False,False,...,False,True,False,False,True,False,True,False,True,False
6603,8690,26.0,0.0,0.0,1872.0,1.0,0.0,True,False,False,...,False,True,False,False,False,True,True,False,False,True
6604,8691,32.0,0.0,1049.0,0.0,353.0,3235.0,False,True,False,...,False,False,False,True,False,False,True,False,True,False


**Perform Train Test Split**

In [143]:
features = spaceship_dummies
target = spaceship_dummies['Spa']  # Reemplaza 'TargetVariable' con el nombre de la columna que contiene el target


In [144]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [145]:
knn = KNeighborsRegressor()

In [146]:
knn.fit(X_train, y_train)

KNeighborsRegressor()

In [147]:
knn.score(X_test, y_test)

0.947868653623878

In [148]:
y_predict = knn.predict(X_test)

In [150]:
mse = mean_squared_error(y_test, y_predict)
mae = mean_absolute_error(y_test, y_predict)
r2 = r2_score(y_test, y_predict)

mse, mae, r2

(64862.18223903177, 67.71180030257186, 0.947868653623878)

- Evaluate your model's performance. Comment it

In [151]:
#vamos a hacer el pca
y_pred_train = knn.predict(X_train)

In [152]:
pca = PCA(n_components = 0.80)
x_pca = pca.fit_transform(X_train)

In [153]:
x_pca_test = pca.transform(X_test)

In [154]:
knn_pca = KNeighborsRegressor()
knn_pca.fit(x_pca, y_train)

KNeighborsRegressor()

In [155]:
y_predict_pca = knn_pca.predict(x_pca_test)

In [156]:
mse_pca = mean_squared_error(y_test, y_predict_pca)
mae_pca = mean_absolute_error(y_test, y_predict_pca)
r2_pca = r2_score(y_test, y_predict_pca)

mse_pca, mae_pca, r2_pca

(530664.1780030257, 218.57745839636917, 0.5734920238895626)

In [157]:
#el mse ha mejorado, pero el mae y el r2 han empeorado.

mse, mae y r2 original: (64862.18223903177, 67.71180030257186, 0.947868653623878)



In [158]:
#estandarizamos el modelo
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)

In [159]:
knn_standardized = KNeighborsRegressor()
knn_standardized.fit(X_train_standardized, y_train)

KNeighborsRegressor()

In [160]:
y_predict_standardized = knn_standardized.predict(X_test_standardized)

In [161]:
mse_standardized = mean_squared_error(y_test, y_predict_standardized)
mae_standardized = mean_absolute_error(y_test, y_predict_standardized)
r2_standardized = r2_score(y_test, y_predict_standardized)

mse_standardized, mae_standardized, r2_standardized

(138864.92317700453, 128.77473524962178, 0.8883908132637928)

mse, mae y r2 original: (64862.18223903177, 67.71180030257186, 0.947868653623878)



In [162]:
#tanto el mse, el mae y el r2 están peor con respecto al original.

In [164]:
#normalizamos el modelo
normalizer = MinMaxScaler()
X_train_normalized = normalizer.fit_transform(X_train)
X_test_normalized = normalizer.transform(X_test)

In [165]:
knn_normalized = KNeighborsRegressor()
knn_normalized.fit(X_train_normalized, y_train)

KNeighborsRegressor()

In [166]:
y_predict_normalized = knn_normalized.predict(X_test_normalized)

In [167]:
mse_normalized = mean_squared_error(y_test, y_predict_normalized)
mae_normalized = mean_absolute_error(y_test, y_predict_normalized)
r2_normalized = r2_score(y_test, y_predict_normalized)

mse_normalized, mae_normalized, r2_normalized

(479165.7629652043, 230.65809379727685, 0.6148825787473116)

mse, mae y r2 original: (64862.18223903177, 67.71180030257186, 0.947868653623878)

In [168]:
#en este último modelo todos los tests dan peores resultados.
#nos quedamos con el modelo sin transformaciones.